In [1]:
import pandas as pd
import numpy as np
import email
import os
from email.parser import Parser
import re
import lxml.html

In [2]:
ham_dir = '/media/sf_Windows/SPAM_ASSASSIN/HAM/'
ham_emails = os.listdir(ham_dir)
spam_dir = '/media/sf_Windows/SPAM_ASSASSIN/SPAM/'
spam_emails = os.listdir(spam_dir)

In [3]:
emails = os.listdir(spam_dir+spam_emails[0]+'/')

In [4]:
ham_dir+ham_emails[0]+'/'+emails[0]

'/media/sf_Windows/SPAM_ASSASSIN/HAM/easy_ham_2002/0494.a0865131f55d26362a8efad99c37de01'

In [20]:
f = open(spam_dir+spam_emails[0]+'/'+emails[43], 'r')
msg = f.read()
f.close()

In [14]:
parser = Parser()
result = parser.parsestr(msg)
tmp = result.get_payload()

In [21]:
msg

'From susan_brown@excite.com  Sun Sep 22 21:56:25 2002\nReturn-Path: <susan_brown@excite.com>\nDelivered-To: zzzz@localhost.jmason.org\nReceived: from localhost (jalapeno [127.0.0.1])\n\tby zzzzason.org (Postfix) with ESMTP id 5AD0D16F03\n\tfor <zzzz@localhost>; Sun, 22 Sep 2002 21:56:24 +0100 (IST)\nReceived: from jalapeno [127.0.0.1]\n\tby localhost with IMAP (fetchmail-5.9.0)\n\tfor zzzz@localhost (single-drop); Sun, 22 Sep 2002 21:56:24 +0100 (IST)\nReceived: from webnote.net (mail.webnote.net [193.120.211.219]) by\n    dogma.slashnull.org (8.11.6/8.11.6) with ESMTP id g8MFWmC06188 for\n    <zzzz4_example.com@jmason.org>; Sun, 22 Sep 2002 16:32:48 +0100\nReceived: from rack3.easydns.com (rack3.easydns.com [205.210.42.50]) by\n    webnote.net (8.9.3/8.9.3) with ESMTP id QAA12471 for <zzzz4@example.com>;\n    Sun, 22 Sep 2002 16:33:21 +0100\nFrom: susan_brown@excite.com\nReceived: from excite.com (unknown [80.48.248.33]) by rack3.easydns.com\n    (Postfix) with SMTP id 404084BB2F for

In [15]:
tmp = re.sub('\n|\t','',tmp)

TypeError: expected string or buffer

In [ ]:
a = lxml.html.fromstring(tmp).text_content()

In [ ]:
a = a.replace(u'\xa0', u' ').replace(u'\xa9',u' ')
robust_decode(a)

In [3]:
##encode
def robust_decode(bs):
    '''Takes a byte string as param and convert it into a unicode one.
First tries UTF8, and fallback to Latin1 if it fails'''
    cr = None
    try:
        #cr = bs.decode('utf8')
        cr = bs.decode('utf-8-sig')
    except UnicodeDecodeError:
        cr = bs.decode('ISO-8859-1')
        cr = cr.encode("ascii", "ignore")
        #cr = cr.decode('utf-8-sig')
    return cr

# extract the body of email
def walkMsg(msg):
    """
    Input: string of email
    Output: body of email
    """
    for part in msg.walk():
        if part.get_content_type() == "multipart/alternative":
            continue
        yield part.get_payload(decode=1)

        # extract the information of email
# still need some works to get number of cc and bcc
def info_of_email(path,List_of_files,classification):
    """
    Input: path, list of files name, classification(spam or ham)
    Ourput: a list with info of email
    """
    parser = Parser()
    info = []
    for files in List_of_files:
        emailText =  open(path + files, 'rb').read()
        email = parser.parsestr(emailText)
        if email.get('cc') == None:
            no_cc = 0
        else:
            no_cc = len(email.get('cc'))
        if email.get("bcc") == None:
            no_bcc = 0
        else:
            no_bcc = len(email.get('bcc'))
         
        Date = email.get("Date")
        special_chars = re.compile(r"\t\n\r")
        
        try: 
            year = re.search('\d{4}',Date).group(0)
        except:
            year = None
        try:
            hour = re.search('\d{2}:\d{2}:\d{2}',Date).group(0).split(":")[0]
        except:
            hour = None
        try: 
            weekday = re.search('Mon|Tue|Wed|Thu|Fri|Sat|Sun',Date).group(0)
        except: 
            weekday =None
        try:
            month = re.search('Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec|Sept',Date).group(0)
        except:
            month =None
        
        Content = " ".join([robust_decode(i) for i in walkMsg(email) if i != None])
        Content = re.sub('\n|\t',' ',Content)
        if 'html' in Content:
                Content = lxml.html.fromstring(Content).text_content() 
        Content = Content.replace(u'\xa0', u' ').replace(u'\xa9',u' ')
        
        
        info.append([year, weekday, hour, month,
                         email.get("From"),email.get('To'),
                         no_cc,no_bcc, 
                         str(email.get("Content-Type")).split(";")[0].lower(),
                         email.get('subject'),
                         Content,
                         classification])
    return info

In [4]:
ham_results = []
for ham_folder in ham_emails:
    emails = os.listdir(ham_dir+ham_folder)
    ham_results.append(info_of_email(ham_dir+ham_folder+'/',emails,'ham'))

In [5]:
hamList = [pd.DataFrame(df,
                      columns=['Year','Weekday',"Hour","Month",
                          'From','To','cc','bcc','Content_type','Subjects','Content','Classification']) for df in ham_results]
hamDF = pd.concat(hamList,ignore_index=True)

In [6]:
spam_results = []
for spam_folder in spam_emails:
    emails = os.listdir(spam_dir+spam_folder)
    spam_results.append(info_of_email(spam_dir+spam_folder+'/',emails,'spam'))

In [7]:
spamList = [pd.DataFrame(df,
                      columns=['Year','Weekday',"Hour","Month",
                          'From','To','cc','bcc','Content_type','Subjects','Content','Classification'])for df in spam_results]
spamDF = pd.concat(spamList,ignore_index=True)

In [68]:
pd.options.display.max_colwidth = 500

In [10]:
from collections import Counter
Counter(spamDF['Year'])

Counter({'2002': 2138})

In [9]:
hamDF = hamDF[hamDF.Year == '2002']
spamDF = spamDF[spamDF.Year == '2002']

In [15]:
spamassassin = pd.concat([hamDF,spamDF])

In [16]:
import pickle
spamassassin.to_pickle("spamassassin.txt")